In [3]:
#loading packages
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import requests
import urllib
from requests_html import HTML
from requests_html import HTMLSession
import getpass
from re import search
import re
from selenium.webdriver.common.by import By
import pandas as pd
import validators


In [4]:
#importing database
df = pd.read_excel (r'Companies.xlsx')
#print (df)
l= [x for x in df['custumer name']]
print(l)

['LARS SPENGELER -', 'SAN PIETRO SERVIZI SRL', 'ILLUMINANDO FIRENZE SRLS', 'SAVI IMMOBILIER', 'SERENA', 'MEYER WERBUNG', 'SISTEMA DESIGN DI CARLI FEDERICO', 'CAFE DALI', 'AUX DELICES DE L ABBAYE', 'MME FLORENCE YOONGRAM', 'ARKASS VERSICHERUNGSMAKLER', 'AERA CONCEPT', 'MAINTECH SANTE', 'CHEVALLIER CONSEIL', 'EARL DU VERNET', 'BUISSONNIERE ACTIVITES GITES SEJOURS', 'ELICICOLTURA UMBRA', 'STAATLICHE BERUFSSCHULE II BAYREUTH', 'ELETTRA SNC DI VISMAN ANDREA E RIODA ANDREA', 'MAIRIE DE BRETTNACH', 'DR. MED.DENT. KNUT KARST ZAHNARZT', 'ANDREAS FLEER', 'BESTATTUNGEN JUNKER GMBH', 'MAURICE JOEL', 'MROTZEK ELEKTRO GMBH', 'STUDIO PULIX GIORGIO S.A.S.', 'A. HILL GMBH', 'DINAMICA BIESSE S.R.L.', 'MR CATALDO LASORSA', 'SERTECO SRL', 'ALPHADI AKADEMIE', 'ASSOCIATION AIDE MENAGERE MILIEU RURAL', 'RISTORANTE AUGUSTA', 'MR N TIZRA', 'CHRISTIAN HEIDER BAUGEWERBE', 'DITTA GIUGLIANI VERONICA', 'MONSIEUR CHRISTOPHE DEBEAULIEU', 'BURKHARD SUDHOFF', 'TOSKANAWORLD', 'MANFRED KANTORSKI', 'MAREIKE MARTIN PD09 PO

In [92]:
def google(query):
    query = urllib.parse.quote_plus(query)
    url = 'https://google.com/search?q=' + query

    # Fetch the URL data using requests.get(url),
    # store it in a variable, request_result.
    request_result=requests.get(url)

    # Creating soup from the fetched request
    soup = bs(request_result.text,"html.parser")
    return(soup)
    #print(soup)
    #for link in soup.find_all('a',attrs={'href': re.compile("^https://")}):
     #   print(link.get('href'))  
    

In [155]:
#linked in 
def linkedin(query):
#turning query to google friendly text ( spaces into plus signs)
    query = query + " linkedin"
    query = urllib.parse.quote_plus(query)
# get html code for said query
    session = HTMLSession()
    response = session.get("https://www.google.co.uk/search?q=" + query)
#get list of links from html code
    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
#remove unwanted links by criteria
    for url in links[:]:
        if url.startswith(google_domains) or (not(search("linkedin",url)) and not(search(query,url)) ) or (search("translate",url))  or not(search("/company/|/school/",url)) :
            links.remove(url)
#return source link/ removes other links
    if(links==[]) :
        return("Linkedin Introuvable")
    else : 
        return(links[0])


In [69]:
#facebook
def facebook(query):
#turning query to google friendly text ( spaces into plus signs)
    query = query + " facebook"
    query = urllib.parse.quote_plus(query)
# get html code for said query
    session = HTMLSession()
    response = session.get("https://www.google.co.uk/search?q=" + query)
#get list of links from html code
    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
#remove unwanted links by criteria
    for url in links[:]:
        if url.startswith(google_domains) or not(search("facebook",url))    :
            links.remove(url)
#return source link/ removes other links
    if(links == []) :
        return("Facebook Introuvable")
    else : 
        return(min(links,key=len))

In [70]:
#instagram
def instagram(query):
#turning query to google friendly text ( spaces into plus signs)
    query = query + "instagram"
    query = urllib.parse.quote_plus(query)
# get html code for said query
    session = HTMLSession()
    response = session.get("https://www.google.co.uk/search?q=" + query)
#get list of links from html code
    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
#remove unwanted links by criteria
    for url in links[:]:
        if url.startswith(google_domains) or (not(search("instagram",url)) and not(search(query,url)) )  :
            links.remove(url)
#return source link/ removes other links
    if(links==[]) :
        return("Instagram Introuvable")
    else : 
        return(min(links,key=len))

In [345]:
def logo(query: str, wd: webdriver, sleep_between_interactions: int = 1):
    
        # build the google query
    max_links_to_fetch = 1
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    i=0
    # load the page
    wd.get(search_url.format(q=query + " logo"))

    image_urls = set()
    image_count = 0
    results_start = 0
    try:
        while image_count < max_links_to_fetch:
            # get all image thumbnail results
            # search by class where css selector= 'img.Q4LuWd' to slect images 
            thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
            number_results = 1
            if thumbnail_results==[] :
                raise Exception

            for img in thumbnail_results[results_start:number_results]:
                # try to click every thumbnail such that we can get the real image behind it
                try:
                    img.click()
                    time.sleep(sleep_between_interactions)
                except Exception:
                    continue

                # extract image urls
                actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
                for actual_image in actual_images:
                    if actual_image.get_attribute('src') :
                        image_urls.add(actual_image.get_attribute('src'))

                image_count = 1

                if len(image_urls) >= max_links_to_fetch:
                    
                    break

    
        return image_urls
     
    except Exception:
        return("Logo Introuvable")

In [346]:
DRIVER_PATH = 'C:\\Program Files\\ChromeDriver\\chromedriver.exe'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
logo("MÃœLLICH SICHERHEITSTECHNIK",wd)

<ipython-input-346-bbdc5a85b508>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)
<ipython-input-345-2bb90ceae271>:17: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


'Logo Introuvable'

In [ ]:
def Website(query):
    url = 'https://google.com/search?q='+urllib.parse.quote_plus(query) #création d'un lien 
    l1=[]
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.')
    request = urllib.request.Request(url) #demande au serveur de google pour avoir la page HTML
    
    request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
    
    raw_response = urllib.request.urlopen(request).read() # Lecture de la page HTML 
    
    html = raw_response.decode("utf-8") #décodage selon UTF-8
    
    soup = BeautifulSoup(html, 'html.parser') # Parser la page HTML
    #print(soup)
    wb=soup.find('a',href=True) # Extraction du lien envers le site 
    #print(wb)
    if (wb!=None and validators.url(wb['href'], public=False)): #controle et stockage dans la nouvelle ligne
        return(wb['href'])
    else : 
        links=soup.find_all('a',href=True)
        #print(links)
        
        for a in links:
            b = str(a)
            if (not(b.startswith(google_domains)) and (search('https://www.',b)) ):
                l1.append(a['href'])
        
        

        #print(l1)
        if(l1==[]) :
            return("Site Web Introuvable")
        else : 
            return(min(l1,key=len))



In [294]:
print(Website("ILLUMINANDO FIRENZE SRLS"))

['https://www.google.com/webhp?hl=ar&sa=X&ved=0ahUKEwiRzIeQt7j3AhXKYMAKHae0DxYQPAgI', 'https://www.lampade.it/plafoniere/', 'https://www.google.com/aclk?sa=l&ai=DChcSEwjzoY6Qt7j3AhVGsO0KHWq6As4YABAEGgJkZw&sig=AOD64_26Af2MK8o-qcEG9Kf7xMTu7E7Neg&ved=2ahUKEwiRzIeQt7j3AhXKYMAKHae0DxYQqyQoAHoECAIQBQ&adurl=', 'https://www.google.com/aclk?sa=l&ai=DChcSEwjzoY6Qt7j3AhVGsO0KHWq6As4YABAFGgJkZw&sig=AOD64_1B2gzR-u76nRpGLBmVU4kYQYzOXg&ved=2ahUKEwiRzIeQt7j3AhXKYMAKHae0DxYQqyQoAXoECAIQBg&adurl=', 'https://www.medialux.it/it/prezzi/illuminando/', 'https://italian.alibaba.com/premium/chandelier_lighting.html', 'https://www.liberoshopping.eu/lampade-da-soffitto/products.1.1.66.sp.uw', 'https://www.illuminando.com/', 'https://www.paginegialle.it/firenze-fi/illuminando-firenze-s-r-l-s', 'https://webcache.googleusercontent.com/search?q=cache:u6WM2GCtHfgJ:https://www.paginegialle.it/firenze-fi/illuminando-firenze-s-r-l-s+&cd=4&hl=ar&ct=clnk&gl=tn', 'https://translate.google.com/translate?hl=ar&sl=it&u=https:

In [348]:
base = pd.DataFrame(columns=['Customer Name','Website','Facebook','Instagram','LinkedIn'])
DRIVER_PATH = 'C:\\Program Files\\ChromeDriver\\chromedriver.exe'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
for i in l :
    row = {"Customer Name":"","Logo":"","Facebook":"","Instagram":"","LinkedIn":""} 
    row["Customer Name"]= i
    row["Logo"] = logo(i,wd)
    row["Facebook"] = facebook(i)
    row["Instagram"] = instagram(i)
    row["Linked in"] = linkedin(i)
    row["Website"]=Website(i)
        
    base = base.append(row,ignore_index=True)
    if (i == 'SARL ARSAPRIM') : 
        break
print(base)
base.to_excel(r'C:\\Users\\Ines Saidane\\OneDrive\\Desktop\\Base_finale.xlsx', index = False)
#PREESNTATION



<ipython-input-348-9ceafa231c5e>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)
<ipython-input-345-2bb90ceae271>:17: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
<ipython-input-345-2bb90ceae271>:31: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


                               Customer Name  \
0                           LARS SPENGELER -   
1                     SAN PIETRO SERVIZI SRL   
2                   ILLUMINANDO FIRENZE SRLS   
3                            SAVI IMMOBILIER   
4                                     SERENA   
..                                       ...   
95  STUDIO LEGALE CUCCIATTI E ASSOCIATI DEGL   
96                              ULRICH BRAUN   
97                   GARAGE ROUSSET CHAVEROT   
98                   ENERGIES ET INNOVATIONS   
99                             SARL ARSAPRIM   

                                              Website  \
0                       https://www.amw-spengeler.de/   
1                           https://www.depagroup.it/   
2                        https://www.illuminando.com/   
3                     https://www.savi-immobilier.fr/   
4                 https://www.magpictures.com/serena/   
..                                                ...   
95  https://www.google.c